In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import coordinates as coord
from astropy import constants, cosmology, units

import cdsed_modelling as colddust_sed_models

from scipy.integrate import quad

import numpy as np
from astropy.io import fits
from astropy.table import Table


from IPython.display import display, Math

In [ ]:
#Insert here the redshift of the source. The unities to be used 
#are specified next to each quantity. Cosmology used is from Planck2018

z = 6.003
cosmop = cosmology.FlatLambdaCDM(H0=67.4, Om0=0.315, Tcmb0=2.726) 

D_l = cosmop.luminosity_distance(z).to('kpc').value #kpc
scale = cosmop.kpc_proper_per_arcmin(z).to('kpc/arcsec').value #kpc/"
c = constants.c.value #m/s
Msun = constants.M_sun.value #kg

#print(D_l, c, scale, Msun)

In [ ]:
#Insert the area of the galaxy in kpc^2. Here, for instance, I put the FWHM sizes in arcsec^2

A = np.array([0.367*0.316,0.355*0.306])*np.pi*scale**2/4 #kpc^2
A = np.mean(A)

#Insert the observed frequency of your observations in GHz. If you want to put the rest-frame frequencies, 
#just adapt the code accordingly, leaving the mae of the variables unchanged

nu_obs = np.array([103.4014,239.2784,247.4335,248.4906,286.8863,325.7194,343.5505,462.8249]) #GHz
nu =  nu_obs*(1+z) #rest frame frequency, GHz
wave_obs = c*1e6/(nu_obs*1e9) #micrometer

#Insert the flux densities correspoding to the observed frequecies above in mJy and thei errors

flux = np.array([0.263,4.313,4.947,4.105,6.622,9.374,7.112,15.46]) #mJy 
err1 = np.array([0.033,0.041,0.062,0.086,0.084,0.086,0.043,0.33])  #mJy

#This is just for taking into account the calibration uncertainties that are added in quadrature 
# to err1 above. If this is not needed, comment the following two lines and rename the above err1 as flux_err

quad1 = np.array([0.02,0.06,0.07,0.07,0.07,0.08,0.09,0.1])
flux_err = np.sqrt(err1**2+(flux*quad1)**2)

# Set the values of fixed paramters according to your needs. Below are some examples.

#fix_par = [50,1.6] #values of fixed parameters if you need to fix [beta, Tdust] --> log(Mdust/Msun) free
fix_par = [50] #if you need to fix Tdust only --> log(Mdust/Msun), beta free
#fix_par = 0 #if you do not need to fix any paramter --> --> log(Mdust/Msun), beta, Tdust free 

#Set the values for the starting position of the chain. It can be randomised.

#start_pos = [8.] #example for [log10(Mdust/Msun)] 
start_pos = [8.5,1.5] #example for [log10(Mdust/Msun), beta] 
#start_pos = [8.,1.5, 60.0] #example for [log10(Mdust/Msun), beta, Tdust] 

#Set number of free paramters. It can be 1 or 2 or 3.

ndim = 2 

#Type of free parameter

if ndim==1:
    part = np.array(['Md']) #np.array(['Td']), np.array(['beta'])
elif ndim==2:
    part = np.array(['Md','beta']) #np.array(['Md','beta']), part = np.array(['Md','Td'])

In [ ]:
#Plot SED

font ='Times New Roman'

freq_plot = np.arange(10,8000) #GHz
#name = 'MACS0416-Y1-b2-T40'

ylim = [2e-5, 60]
xlim = [20,7000]
pos_text = [350,6]

wave_plot = c*1e6/(freq_plot*1e9) #micrometri

yd, yu = ylim
xd, xu = xlim

fig, ax = plt.subplots(figsize=(11,9))
ax.set_xscale('log')
ax.set_yscale('log')
ax.tick_params(axis = 'both', which = 'major', direction = 'in', length = 15, color = 'k', top = True, right = True, labelsize = 15, width = 1.2)
ax.tick_params(axis = 'both', which = 'minor', direction = 'in', length = 5, top = True, right = True,width = 1.2)
plt.ylim(yd, yu)
ax.set_xlim(xd,xu)

ax.errorbar(nu_obs,flux, yerr = flux_err, capsize = 3, lw = 1.4, marker = 'o', ms =6, ls='none',color = 'cyan', markeredgecolor='mediumblue',ecolor='mediumblue', zorder =3)
model2 = colddust_sed_models.results_plot.sed_func_plot(freq_plot,nu_obs,part,[7.4,2.5],ndim,z,A,D_l,50)
model3 = colddust_sed_models.results_plot.sed_func_plot(freq_plot,nu_obs,part,[7.6,1.8],ndim,z,A,D_l,50)

ax.plot(freq_plot,model2, label = 'MBB', c = 'red', lw = 1.5, zorder =1)
ax.plot(freq_plot,model3, label = 'MBB', c = 'gray', ls='--',lw = 1.5, zorder =1)

In [ ]:
#Here the chain is runned or imported (if previously runned)
#Always remember to remove the file .h5 if you want to run twice the same chain.

filename = 'j183-sed-MCMC.h5' #change the filename as you wish. Here the chain will be saved

#For additional inputs in run_chain() see Readme.md
sampler = colddust_sed_models.sed_models.run_chain(start_pos, ndim,part,filename,nu_obs,\
                                                   flux,flux_err,z,D_l,A,fix_par,iteration=1000,run=True) 
print('Chain done')

In [ ]:
#Print the best-fitting values

#popt = best-fitting [log(Mdust/Msun), beta, Tdust] in case ndim=3
#q = down and up error for popt
#mdust = [Mdust, err_down_Mdust, err_up_Mdust]

popt, q, mdust = colddust_sed_models.results_plot.sed_results(sampler, ndim, dis=250)

label_new = ["\log(M_{dust}/M_\odot)","beta"] 
#label_new = ["\log(M_{dust}/M_\odot)"] 
j = 0
for i in range(ndim):
    txt = "\mathrm{{{3}}} = {0:.3f}_{{-{1:.3f}}}^{{+{2:.3f}}}"
    txt2 = txt.format(popt[i], q[j], q[j+1], label_new[i])
    display(Math(txt2))
    j = j+2

print('M_dust/M_sun = %.2E - %.2E + %.2E' %(mdust[0], mdust[1], mdust[2]))

In [ ]:
#Plot the chains and the corner plot

name = 'T50-beta16' 
colddust_sed_models.results_plot.sed_res_plot(sampler, ndim, name, part,save=False)

In [ ]:
#Plot SED

freq_plot = np.arange(10,8000) #GHz
name_plot = 'QSO' 

ylim = [2e-5, 16]
xlim = [20,7000]
pos_text = [350,1]

colddust_sed_models.results_plot.plot_sed(freq_plot,sampler,part, nu_obs,flux,flux_err,z,D_l,A,c,fix_par,\
                                          ndim,ylim,xlim,pos_text,name_plot,name,\
                                          display_res=True,save=False)


In [ ]:
#Compute far-infrared luminosity (40-1000 microm) and associated SFR (using Chabrier IMF)

LFIR, SFR = colddust_sed_models.results_plot.lfir_sfr(popt,ndim,z,c,D_l,A,fix_par)

print('FIR Luminosity : %.3E L_sun' %(LFIR))
print('SFR: %.2f M_sun/yr' %SFR)

#Compute total-infrared luminosity (8-1000 microm) and associated SFR (using Chabrier IMF)

LTIR, SFR2 = colddust_sed_models.results_plot.ltir_sfr(popt,ndim,z,c,D_l,A,fix_par)

print('TIR Luminosity : %.3E L_sun' %(LTIR))
print('SFR: %.2f M_sun/yr' %SFR2)